In [6]:
import taxoniq

mammals = {}
non_mammals = {}
not_found = set()

f = open('../clustering98_accessions.txt')

for line in f.readlines():
    acc = line.split(">")[1].split()[0]
    name = " ".join(line.split(">")[1].replace(" UNVERIFIED: ", " ").split()[1:3])

    try:
        t = taxoniq.Taxon(scientific_name=name)
        ranks = [(t.rank.name, t.scientific_name) for t in t.ranked_lineage]
        species = ''
        genus = ''
        for rank in ranks:
            if rank[0] == 'species':
                species = rank[1]
            if rank[0] == 'genus':
                genus = rank[1]

        if ('class', 'Mammalia') in ranks:
            mammals[acc] = species
        else:
            non_mammals[acc] = species

    except:
        not_found.add(line.strip())
f.close()

In [7]:
import gzip
from Bio import SeqIO

genomes = {}
with gzip.open('../mitochondria_dereplicated98.fasta.gz', 'rt') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        # Process each record as needed
        genomes[record.id] = record.seq


In [12]:
def mutate_sequence(seq, mutation_rate):
    nucleotides = ['A', 'C', 'G', 'T']
    seq_list = list(seq)
    
    num_mutations = int(len(seq) * mutation_rate)
    
    for _ in range(num_mutations):
        position = random.randint(0, len(seq) - 1)
        # Select a new nucleotide that is different from the current one
        new_nucleotide = random.choice([n for n in nucleotides if n != seq_list[position]])
        seq_list[position] = new_nucleotide
    
    mutated_seq = ''.join(seq_list)
    
    return mutated_seq


In [16]:
mutate = [0,0.01,0.03,0.05]
iterations = 100
import pandas as pd
import random

for i in range(0, iterations):
    mammal_sample = random.sample(sorted(mammals), 20)
    non_mammal_sample = random.sample(sorted(non_mammals), 40)
    
    for m in mutate:
        f = open("mt_simulated_m" + str(m) + "_"  + str(i) + ".fasta", "w")
        for acc in mammal_sample:
            seq = mutate_sequence(genomes[acc], m)
            f.write(">" + acc + "_" + mammals[acc].replace(" ","_") + "\n")
            f.write(seq + "\n")
        for acc in non_mammal_sample:
            seq = mutate_sequence(genomes[acc], m)
            f.write(">" + acc + "_" + non_mammals[acc].replace(" ","_") + "\n")
            f.write(seq + "\n")
        f.close()

In [10]:
mammals

{'NC_035621.1': 'Rattus baluensis',
 'NC_008426.1': 'Erignathus barbatus',
 'NC_035503.1': 'Neodon sikimensis',
 'JF681039.1': 'Sotalia guianensis',
 'NC_020712.1': 'Hippotragus equinus',
 'NC_053815.1': 'Golunda ellioti',
 'NC_050031.1': 'Microsciurus sabanillae',
 'NC_022424.1': 'Lophostoma silvicolum',
 'NC_020679.1': 'Antilocapra americana',
 'NC_036319.1': 'Myotis yumanensis',
 'NC_064150.1': 'Aotus trivirgatus',
 'MG983792.1': 'Sorex isodon',
 'KR059226.1': 'Capra aegagrus',
 'NC_035655.1': 'Cheirogaleus major',
 'NC_050682.1': 'Callithrix aurita',
 'NC_006901.1': 'Colobus guereza',
 'KY425609.1': 'Trachypithecus phayrei',
 'NC_013068.1': 'Tscherskia triton',
 'KC516778.1': 'Uropsilus sp.',
 'NC_007393.1': 'Rousettus aegyptiacus',
 'JN632673.1': 'Odocoileus virginianus',
 'NC_020644.1': 'Melogale moschata',
 'NC_061537.1': 'Balionycteris maculata',
 'NC_065071.1': 'Hylomyscus aeta',
 'CM024384.1': 'Pipistrellus kuhlii',
 'NC_046485.1': 'Hydrictis maculicollis',
 'KY117566.1': 'Ma